# Load dataset


In [7]:
!pip install joblib
!pip install scikit-learn
!pip install pandas
!pip install numpy
from main import ensemble_pipeline
import joblib
joblib.dump(ensemble_pipeline, 'ensemble_pipeline.pkl')
from main import ensemble_pipeline
from DATASET import clean_df
import numpy as np

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]
                              Estudi Curs acadèmic  \
0  Graduat en Enginyeria Informàtica       2020/21   
1  Graduat en Enginyeria Informàtica       2020/21   
2  Graduat en Enginyeria Informàtica       2020/21   
3  Graduat en Enginyeria Informàtica       2020/21   
4  Graduat en Enginyeria Informàtica       2020/21   

                          Id Anonim  Sexe                 Assignatura  \
0  1DFB71F2B000D1421808D0B3F67B335E  Home                     Àlgebra   
1  1DFB71F2B00

In [8]:
df = clean_df.copy()
df = df.dropna(subset=['nota_assignatura'])

# Separate current year data (to predict)
df_train = df[df['curs_academic'] != '2023/24'].copy()
df_pred_target = df[df['curs_academic'] == '2023/24'].copy()

#Define target variable (Nota_assignatura) and input features (droping target from features)
X_train = df_train.drop(columns=['nota_assignatura'])
y_train = df_train['nota_assignatura']

X_pred = df_pred_target.drop(columns=['nota_assignatura'])
y_pred = df_pred_target['nota_assignatura']   # Ground truth for 2023/24

ensemble_pipeline.fit(X_train, y_train)

df_pred_target['predicted_nota_assignatura'] = ensemble_pipeline.predict(X_pred)


# Predictions 


In [11]:
id_alumne_random = np.random.choice(df_pred_target['id_anonim'].unique())
df_alumne_random = df_pred_target[df_pred_target['id_anonim'] == id_alumne_random]
df_alumne_random_resultados = df_alumne_random[['id_anonim', 'assignatura', 'curs_academic', 'predicted_nota_assignatura', 'nota_assignatura']]

# Imprimir los resultados
print(f"Resultados para el alumno con id_anonim {id_alumne_random}:")
print(df_alumne_random_resultados)

Resultados para el alumno con id_anonim 5570280cd3e392c38bda8c7e90641f00:
                              id_anonim                  assignatura  \
27579  5570280cd3e392c38bda8c7e90641f00  Arquitectura de computadors   
27580  5570280cd3e392c38bda8c7e90641f00               Bases de dades   
27581  5570280cd3e392c38bda8c7e90641f00      Enginyeria del software   
27582  5570280cd3e392c38bda8c7e90641f00                  Estadística   
27583  5570280cd3e392c38bda8c7e90641f00    Estructura de computadors   
27584  5570280cd3e392c38bda8c7e90641f00       Informació i seguretat   
27585  5570280cd3e392c38bda8c7e90641f00    Intel·ligència artificial   
27586  5570280cd3e392c38bda8c7e90641f00    Laboratori de programació   
27587  5570280cd3e392c38bda8c7e90641f00           Sistemes operatius   
27588  5570280cd3e392c38bda8c7e90641f00                       Xarxes   

      curs_academic  predicted_nota_assignatura  nota_assignatura  
27579       2023/24                    6.472767               5.7

First we separated the notes by rank of priority. 
- First we go through the students that would fail (<5)

In [12]:
asignaturas_probable_suspenso = df_alumne_random_resultados[df_alumne_random_resultados['predicted_nota_assignatura'] < 5]

print(f"\nAsignaturas que el modelo predice que suspenderá el alumno {id_alumne_random}:")
print(asignaturas_probable_suspenso[['assignatura', 'predicted_nota_assignatura']])



Asignaturas que el modelo predice que suspenderá el alumno 5570280cd3e392c38bda8c7e90641f00:
      assignatura  predicted_nota_assignatura
27588      Xarxes                    4.645085


Possible messages for all the students:
De momento hace un mensaje para los que hay mucha difernecia entre nota (quitar) y los que tienen riesgo a suspender.

In [ ]:
informes = []

SUSPENSO = 5
DIFFERENCE = 2


for alumno_id in df_pred_target['id_anonim'].unique():
    alumno_data = df_pred_target[df_pred_target['id_anonim'] == alumno_id].copy()
    alumno_data['alerta'] = ""

    for idx, row in alumno_data.iterrows():
        pred = row['predicted_nota_assignatura']
        real = row['nota_assignatura']
        asignatura = row['assignatura']

        if pred < SUSPENSO:
            alumno_data.at[idx, 'alerta'] = f" Posible suspenso en '{asignatura}' (Predicho: {pred:.2f})"
        elif abs(pred - real) > DIFFERENCE:
            alumno_data.at[idx, 'alerta'] = f" Alta diferencia entre real y predicho en '{asignatura}' (Predicho: {pred:.2f}, Real: {real:.2f})"

    # Guardar informe solo si hay alertas
    alertas_alumno = alumno_data[alumno_data['alerta'] != ""].copy()
    if not alertas_alumno.empty:
        alertas_alumno['id_anonim'] = alumno_id  # AÑADIMOS EL ID
        informes.append((alumno_id, alertas_alumno[['id_anonim', 'assignatura', 'predicted_nota_assignatura', 'nota_assignatura', 'alerta']]))


for alumno_id, alertas in informes[:5]:  # Mostramos solo los 5 primeros alumnos
    print(f"\n Informe para alumno: {alumno_id}")
    print(alertas.to_string(index=False))


 Informe para alumno: 0f45e92fec9ab96236dc5ad7f96713e2
                       id_anonim                      assignatura  predicted_nota_assignatura  nota_assignatura                                                                   alerta
0f45e92fec9ab96236dc5ad7f96713e2 Organització i gestió d'empreses                    4.656194               5.0  Posible suspenso en 'Organització i gestió d'empreses' (Predicho: 4.66)

 Informe para alumno: F4537a70eeeafbae2094f3ad4a203f24
                       id_anonim           assignatura  predicted_nota_assignatura  nota_assignatura                                                                                         alerta
F4537a70eeeafbae2094f3ad4a203f24 Anglès professional i                    6.636199               8.7  Alta diferencia entre real y predicho en 'Anglès professional i' (Predicho: 6.64, Real: 8.70)

 Informe para alumno: 3cb18b97ef73da586e4a041bb449c613
                       id_anonim                      assignatura  pre

Concatenar y guardar

In [19]:
!pip install streamlit
import streamlit as st
import pandas as pd

alertas_totales = pd.concat([alertas for _, alertas in informes], ignore_index=True)
alertas_totales.to_csv('alertas_academicas.csv', index=False)

# Mostrar en Streamlit
df_alertas = pd.read_csv('alertas_academicas.csv')
st.title("Alertas Académicas")
st.dataframe(df_alertas)

2025-05-06 13:14:01.525 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 13:14:01.525 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 13:14:01.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-06 13:14:01.532 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()